In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import figure_factory as figfac
import sklearn 


###  Reading Data 



In [2]:
df = pd.read_csv('../Datasets/Gold Price (2013-2023).csv')
df.head(10)


,Date,Price,Open,High,Low,Vol.,Change %
0,12/30/2022,"1,826.20","1,821.80","1,832.40","1,819.80",107.50K,0.01%
1,12/29/2022,"1,826.00","1,812.30","1,827.30","1,811.20",105.99K,0.56%
2,12/28/2022,"1,815.80","1,822.40","1,822.80","1,804.20",118.08K,-0.40%
3,12/27/2022,"1,823.10","1,808.20","1,841.90","1,808.00",159.62K,0.74%
4,12/26/2022,"1,809.70","1,805.80","1,811.95","1,805.55",NaN,0.30%
5,12/23/2022,"1,804.20","1,801.00","1,812.20","1,798.90",105.46K,0.50%
6,12/22/2022,"1,795.30","1,823.80","1,829.30","1,792.70",175.77K,-1.65%
7,12/21/2022,"1,825.40","1,827.90","1,833.80","1,821.30",110.18K,0.00%
8,12/20/2022,"1,825.40","1,796.80","1,832.40","1,793.70",197.50K,1.54%
9,12/19/2022,"1,797.70","1,801.60","1,808.60","1,793.20",86.09K,-0.14%


#### As you can see, the data set includes daily gold price information including daily Open, High and Low prices and the final price of each day (Price) along with the volume of transactions and price changes in each day

### Dataset basic information:

In [3]:
def check_df(df, head=5):
    print("#################### Shape #################### ")
    print(df.shape)
    print("#################### Types #################### ")
    print(df.dtypes)
    print("#################### Head #################### ")
    print(df.head(head))

    print("#################### Describe #################### ")
    print(df.describe().T)
    print("#################### NA #################### ")
    print()
    print("#################### Columns #################### ")
    print(df.columns)


In [4]:
check_df(df)

#################### Shape #################### 
(2583, 7)
#################### Types #################### 
Date        object
Price       object
Open        object
High        object
Low         object
Vol.        object
Change %    object
dtype: object
#################### Head #################### 
         Date     Price      Open      High       Low     Vol. Change %
0  12/30/2022  1,826.20  1,821.80  1,832.40  1,819.80  107.50K    0.01%
1  12/29/2022  1,826.00  1,812.30  1,827.30  1,811.20  105.99K    0.56%
2  12/28/2022  1,815.80  1,822.40  1,822.80  1,804.20  118.08K   -0.40%
3  12/27/2022  1,823.10  1,808.20  1,841.90  1,808.00  159.62K    0.74%
4  12/26/2022  1,809.70  1,805.80  1,811.95  1,805.55      NaN    0.30%
#################### Describe #################### 
         count unique         top freq
Date      2583   2583  12/30/2022    1
Price     2583   2072    1,294.30    5
Open      2583   2061    1,284.00    5
High      2583   2044    1,220.00    6
Low       2583   2

### Data peraparation:
 

  1. Features subject selection 

In [5]:
df.drop(['Vol.', 'Change %'], axis=1, inplace=True)


In [6]:
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)


In [7]:
NumCols = df.columns.drop(['Date'])
df[NumCols] = df[NumCols].replace({',': ''}, regex=True)
df[NumCols] = df[NumCols].astype('float64')


In [8]:
df.head ()


,Date,Price,Open,High,Low
0,2013-01-02,1689.9,1675.8,1695.0,1672.1
1,2013-01-03,1675.6,1688.0,1689.3,1664.3
2,2013-01-04,1649.9,1664.4,1664.4,1630.0
3,2013-01-07,1647.2,1657.3,1663.8,1645.3
4,2013-01-08,1663.2,1651.5,1662.6,1648.8


We need to change the date variable

In [9]:
df.isnull().sum().sum()



0

In [10]:
df.duplicated().head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [11]:
print("Shape of the dataset: ", df.shape)
print("Length of the dataset: ", len(df))
print("Minimum closing price of gold during last then  years: ", df["Low"].min())
print("Maximum closing price of gold during last then  years: ",df["High"].max())


Shape of the dataset:  (2583, 5)
Length of the dataset:  2583
Minimum closing price of gold during last then  years:  1045.4
Maximum closing price of gold during last then  years:  2078.7


In [12]:
fig = px.line(y=df.Price, x=df.Date)
fig.update_traces(line_color='black') 
fig.update_layout(xaxis_title="Date", 
                  yaxis_title="Scaled Price",
                  title={'text': "Gold Price History Data ($)", 'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
                  plot_bgcolor='rgba(200,200,0,04)')

/Users/jack/Desktop/Desktop/ML_Course/.venv/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


### Linear Regression

It is obvious that time series forecasting methods should be used in this case. As it can be seen on the line chart above, the gold prices have positive trend since 2013. Hence regression model is a good method to make predictions

In [ ]:

# transform data set date values to time series indices
def TransformDateToIndex():
    counter = 1
    for x in df['Date']:
        df['Date'].iat[counter - 1] = counter
        counter = counter + 1
TransformDateToIndex()

# reshape dataset axes in order to make regression properly
X_values = df['Date'].values.reshape(-1, 1)
Y_values = df['Value'].values.reshape(-1, 1)

# split the dataset for training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X_values, Y_values, test_size = 0.2)

# create linear regression instance
regression = LinearRegression()

# fit linear regression line using training data
regression.fit(X_train, Y_train)

# make predictions from independent test variable
Y_pred = regression.predict(X_test)

# visualize raw data and regression model on same graph
pyplot.plot(df['Date'], df['Value'])
pyplot.plot(X_test, Y_pred)
pyplot.title("Linear Regression Model")
pyplot.xlabel("Date")
pyplot.ylabel("Value")
pyplot.show()

# show intercept value of regression line
print("intercept:", regression.intercept_)

# show slope value of regression line
print("slope:", regression.coef_)
print("\n")

# show forecasting model metrics for performance evaluation
print("Mean Absolute Error: ", metrics.mean_absolute_error(Y_test, Y_pred))
print("R-Squared: ", metrics.r2_score(Y_test, Y_pred))
print("\n")

# show actual test data and predicted data
ResultsDataFrame = pd.DataFrame({'Actual Data' : Y_test.squeeze(), 'Predicted Data' : Y_pred.squeeze()})
print(ResultsDataFrame)